In [1]:
import json, requests
import pandas as pd
import numpy as np
import io
import random
import time
from sklearn.pipeline import make_pipeline

from joblib import dump, load

from random import sample
import warnings
warnings.filterwarnings('ignore')

In [2]:
# load test data
df = pd.read_csv('test_model_input.csv')
# create column set
features = ['return', 'vol', 'mom', 'sma', 'min', 'max']  
lags = 6

cols = []
for f in features:
  for lag in range(1, lags + 1):
    col = f'{f}_lag_{lag}'
    cols.append(col)

### Test Vanilla hit to endpoint

Try with string as input

In [169]:
mod_input_ex1 = '{"accessKey":"mytbobsrpwzf5nhnqg32nsuqg06n0x7c","request":{"return_lag_1": -0.0021425126979011, "return_lag_2": 0.0011643342299438, \
 "return_lag_3": 0.0005597566804979, "return_lag_4": 0.0018895733736993, "return_lag_5": -0.0014294624692999, "return_lag_6": 0.0001499935149684, \
 "vol_lag_1": 0.0021610100401411, "vol_lag_2": 0.0022901628748157, "vol_lag_3": 0.0022798462625308, "vol_lag_4": 0.0023072489270228, \
 "vol_lag_5": 0.0022711289934632, "vol_lag_6": 0.002332806971207, "mom_lag_1": -1.0, "mom_lag_2": 1.0, "mom_lag_3": -1.0, \
 "mom_lag_4": 1.0, "mom_lag_5": -1.0, "mom_lag_6": -1.0, "sma_lag_1": 4807.098706054688, "sma_lag_2": 4808.402197265625, \
 "sma_lag_3": 4808.28720703125, "sma_lag_4": 4808.2912109375, "sma_lag_5": 4808.030712890625,"sma_lag_6": 4808.390307617187, \
 "min_lag_1": 4795.140625, "min_lag_2": 4795.140625, "min_lag_3": 4795.140625, "min_lag_4": 4795.140625, "min_lag_5": 4795.140625, \
 "min_lag_6": 4799.0, "max_lag_1": 4820.0, "max_lag_2": 4828.27001953125, "max_lag_3": 4828.27001953125, "max_lag_4": 4828.27001953125, \
 "max_lag_5": 4828.27001953125, "max_lag_6": 4828.27001953125}}'
r = requests.post('https://modelservice.ml-6e89be47-314.loki-cdp.a465-9q4k.cloudera.site/model', \
                  data= mod_input_ex1, headers={'Content-Type': 'application/json'})

In [170]:
r.json()

{'success': True,
 'response': {'model_deployment_crn': 'crn:cdp:ml:us-west-1:558bc1d2-8867-4357-8524-311d51259233:workspace:61120f1b-0e6a-483d-a73d-6594c1f7ae6f/e43a7075-96d8-4be2-8de5-55f17d735a8e',
  'prediction': [-1],
  'uuid': 'fa13632f-22c0-4095-9637-f7d8d0b04785'},
 'ReplicaID': 'rf-v2-5-5-58b785b848-dgcxc',
 'Size': 260,
 'StatusCode': 200}

Try second model deployed

In [171]:
mod_input_ex2 = '{"accessKey":"mi0xk4gzgt1ollxjtgsxno785rxiu53l","request":{"return_lag_1": -0.0021425126979011, "return_lag_2": 0.0011643342299438, \
 "return_lag_3": 0.0005597566804979, "return_lag_4": 0.0018895733736993, "return_lag_5": -0.0014294624692999, "return_lag_6": 0.0001499935149684, \
 "vol_lag_1": 0.0021610100401411, "vol_lag_2": 0.0022901628748157, "vol_lag_3": 0.0022798462625308, "vol_lag_4": 0.0023072489270228, \
 "vol_lag_5": 0.0022711289934632, "vol_lag_6": 0.002332806971207, "mom_lag_1": -1.0, "mom_lag_2": 1.0, "mom_lag_3": -1.0, \
 "mom_lag_4": 1.0, "mom_lag_5": -1.0, "mom_lag_6": -1.0, "sma_lag_1": 4807.098706054688, "sma_lag_2": 4808.402197265625, \
 "sma_lag_3": 4808.28720703125, "sma_lag_4": 4808.2912109375, "sma_lag_5": 4808.030712890625,"sma_lag_6": 4808.390307617187, \
 "min_lag_1": 4795.140625, "min_lag_2": 4795.140625, "min_lag_3": 4795.140625, "min_lag_4": 4795.140625, "min_lag_5": 4795.140625, \
 "min_lag_6": 4799.0, "max_lag_1": 4820.0, "max_lag_2": 4828.27001953125, "max_lag_3": 4828.27001953125, "max_lag_4": 4828.27001953125, \
 "max_lag_5": 4828.27001953125, "max_lag_6": 4828.27001953125}}'
r = requests.post('https://modelservice.ml-6e89be47-314.loki-cdp.a465-9q4k.cloudera.site/model', \
                  data= mod_input_ex2, headers={'Content-Type': 'application/json'})

In [172]:
r.json()

{'success': False,
 'ReplicaID': 'rf-new-revision-6-7-5fcb7586cb-bs4sb',
 'Size': 101,
 'StatusCode': 400}

### Automate the dictionary build

In [199]:
# Model - new model- 
model_url = 'https://modelservice.ml-6e89be47-314.loki-cdp.a465-9q4k.cloudera.site/model'
accessKey = 'mi0xk4gzgt1ollxjtgsxno785rxiu53l'

# Model - prior model - 
#model_url = 'https://modelservice.ml-6e89be47-314.loki-cdp.a465-9q4k.cloudera.site/model'
#accessKey = 'mytbobsrpwzf5nhnqg32nsuqg06n0x7c'

In [200]:
input_data_single = df[cols].head(1).to_dict('records')[0]

In [201]:
#input_data_single

In [202]:
# build embedded dictionary step 1    
request_dict = {"request":input_data_single}

# access key will be end point specific
BackDict = {"accessKey":accessKey}
BackDict.update(request_dict)
request_dict=BackDict

In [203]:
request_dict

{'accessKey': 'mi0xk4gzgt1ollxjtgsxno785rxiu53l',
 'request': {'return_lag_1': -0.0021425126979011,
  'return_lag_2': 0.0011643342299438,
  'return_lag_3': 0.0005597566804979,
  'return_lag_4': 0.0018895733736993,
  'return_lag_5': -0.0014294624692999,
  'return_lag_6': 0.0001499935149684,
  'vol_lag_1': 0.0021610100401411,
  'vol_lag_2': 0.0022901628748157,
  'vol_lag_3': 0.0022798462625308,
  'vol_lag_4': 0.0023072489270228,
  'vol_lag_5': 0.0022711289934632,
  'vol_lag_6': 0.002332806971207,
  'mom_lag_1': -1.0,
  'mom_lag_2': 1.0,
  'mom_lag_3': -1.0,
  'mom_lag_4': 1.0,
  'mom_lag_5': -1.0,
  'mom_lag_6': -1.0,
  'sma_lag_1': 4807.098706054688,
  'sma_lag_2': 4808.402197265625,
  'sma_lag_3': 4808.28720703125,
  'sma_lag_4': 4808.2912109375,
  'sma_lag_5': 4808.030712890625,
  'sma_lag_6': 4808.390307617187,
  'min_lag_1': 4795.140625,
  'min_lag_2': 4795.140625,
  'min_lag_3': 4795.140625,
  'min_lag_4': 4795.140625,
  'min_lag_5': 4795.140625,
  'min_lag_6': 4799.0,
  'max_lag_1

In [206]:
r = requests.post(model_url, \
    data=json.dumps(request_dict), \
        headers={'Content-Type': 'application/json'})

In [207]:
r.json()

{'success': True,
 'response': {'model_deployment_crn': 'crn:cdp:ml:us-west-1:558bc1d2-8867-4357-8524-311d51259233:workspace:61120f1b-0e6a-483d-a73d-6594c1f7ae6f/7ab1aaf0-800b-4548-91a3-09f835a9374b',
  'prediction': [-1],
  'uuid': 'd22ca6e8-d46c-4386-b969-3f9ab3bde7fc'},
 'ReplicaID': 'rf-new-revision-6-9-787b9d7447-5vf84',
 'Size': 260,
 'StatusCode': 200}

### Try multiple

In [214]:
accessKey = 'm11sdyxpwyfdnbe1cf45keu9q260j765'
accessKey

'm11sdyxpwyfdnbe1cf45keu9q260j765'

In [215]:
input_data_mult = dict()
for col in cols:
    input_data_mult[col] = df[col].to_list()    
#    input_data_mult[col] = df[col].head(1).to_list()    

# build embedded dictionary step 1    
request_dict = {"request":input_data_mult}

# access key will be end point specific
BackDict = {"accessKey":accessKey}
BackDict.update(request_dict)
request_dict=BackDict

In [216]:
request_dict

{'accessKey': 'm11sdyxpwyfdnbe1cf45keu9q260j765',
 'request': {'return_lag_1': [-0.0021425126979011,
   -0.0022490930906144,
   -0.0001389282088331,
   -0.000327179097279,
   -0.0007702210963869,
   -0.0014033376973431,
   -0.0003079100537979,
   -0.0005713146543069,
   -0.0029099585021106,
   -0.0005386301292891],
  'return_lag_2': [0.0011643342299438,
   0.0003135160193135,
   -0.0008329524237045,
   0.0013092966822597,
   -0.0006064255431239,
   -0.0014013711001281,
   -0.0027207436866113,
   -0.0015219763920018,
   -0.0003395639922771,
   0.0013473254209539],
  'return_lag_3': [0.0005597566804979,
   0.0008365767857185,
   -0.0009709369976002,
   0.0022955118273249,
   0.0010262245993342,
   0.0008278720489438,
   -0.0001074580728349,
   0.0013316106843146,
   0.0006784368987151,
   0.0002696418989847],
  'return_lag_4': [0.0018895733736993,
   0.0003138768558832,
   0.0001385720187442,
   0.0023007305655657,
   -0.0019911349433234,
   0.0021045371781046,
   -0.0011351459559232,
  

In [217]:
r9 = requests.post(model_url, data=json.dumps(request_dict),headers={'Content-Type': 'application/json'})

In [218]:
r9.json()

{'success': False,
 'ReplicaID': 'rf-direction-3-7-10-6bf48bbf6d-5dgjh',
 'Size': 200,
 'StatusCode': 400}

In [221]:
json.dumps(request_dict)

'{"accessKey": "m11sdyxpwyfdnbe1cf45keu9q260j765", "request": {"return_lag_1": [-0.0021425126979011, -0.0022490930906144, -0.0001389282088331, -0.000327179097279, -0.0007702210963869, -0.0014033376973431, -0.0003079100537979, -0.0005713146543069, -0.0029099585021106, -0.0005386301292891], "return_lag_2": [0.0011643342299438, 0.0003135160193135, -0.0008329524237045, 0.0013092966822597, -0.0006064255431239, -0.0014013711001281, -0.0027207436866113, -0.0015219763920018, -0.0003395639922771, 0.0013473254209539], "return_lag_3": [0.0005597566804979, 0.0008365767857185, -0.0009709369976002, 0.0022955118273249, 0.0010262245993342, 0.0008278720489438, -0.0001074580728349, 0.0013316106843146, 0.0006784368987151, 0.0002696418989847], "return_lag_4": [0.0018895733736993, 0.0003138768558832, 0.0001385720187442, 0.0023007305655657, -0.0019911349433234, 0.0021045371781046, -0.0011351459559232, -0.0010569708475889, 0.0002273234368903, 0.00188984108876], "return_lag_5": [-0.0014294624692999, 0.0010469

request_dict.keys()

In [220]:
input_data_mult.keys()

dict_keys(['return_lag_1', 'return_lag_2', 'return_lag_3', 'return_lag_4', 'return_lag_5', 'return_lag_6', 'vol_lag_1', 'vol_lag_2', 'vol_lag_3', 'vol_lag_4', 'vol_lag_5', 'vol_lag_6', 'mom_lag_1', 'mom_lag_2', 'mom_lag_3', 'mom_lag_4', 'mom_lag_5', 'mom_lag_6', 'sma_lag_1', 'sma_lag_2', 'sma_lag_3', 'sma_lag_4', 'sma_lag_5', 'sma_lag_6', 'min_lag_1', 'min_lag_2', 'min_lag_3', 'min_lag_4', 'min_lag_5', 'min_lag_6', 'max_lag_1', 'max_lag_2', 'max_lag_3', 'max_lag_4', 'max_lag_5', 'max_lag_6'])

In [138]:
# just the prediction
#r.json()['response']['prediction']

The prediction is a list. In this case a single input was given so a list of 1 element is returned

### Test automating input prep for hitting api
1. First try with single row, 
2. Then try with multiple inputs

In [114]:
pd.DataFrame.from_dict(input_data_mult)
#request_dict

,return_lag_1,return_lag_2,return_lag_3,return_lag_4,return_lag_5,return_lag_6,vol_lag_1,vol_lag_2,vol_lag_3,vol_lag_4,...,min_lag_3,min_lag_4,min_lag_5,min_lag_6,max_lag_1,max_lag_2,max_lag_3,max_lag_4,max_lag_5,max_lag_6
0,-0.002143,0.001164,0.000560,0.001890,-0.001429,0.000150,0.002161,0.002290,0.002280,0.002307,...,4795.140625,4795.140625,4795.140625,4799.000000,4820.000000,4828.270020,4828.270020,4828.270020,4828.270020,4828.270020
1,-0.002249,0.000314,0.000837,0.000314,0.001047,-0.000419,0.001329,0.001562,0.001652,0.001652,...,95.309998,95.129997,95.050003,95.044998,95.720001,95.779999,95.779999,95.779999,95.779999,95.779999
2,-0.000139,-0.000833,-0.000971,0.000139,-0.001178,-0.002144,0.000985,0.001016,0.001018,0.001014,...,72.059998,72.120003,72.120003,72.205002,72.660004,72.660004,72.660004,72.660004,72.660004,72.660004
3,-0.000327,0.001309,0.002296,0.002301,0.001317,0.000330,0.001410,0.001586,0.001569,0.001488,...,60.590000,60.590000,60.590000,60.590000,61.139999,61.139999,61.060001,61.040001,61.040001,61.040001
4,-0.000770,-0.000606,0.001026,-0.001991,0.001882,-0.000555,0.001360,0.001828,0.001876,0.001872,...,412.959991,412.959991,412.959991,412.959991,416.480011,416.739990,416.739990,416.739990,416.739990,416.739990
5,-0.001403,-0.001401,0.000828,0.002105,0.000894,-0.002808,0.001383,0.001339,0.001292,0.001289,...,78.014999,78.014999,78.010002,78.010002,78.550003,78.550003,78.550003,78.485001,78.470001,78.470001
6,-0.000308,-0.002721,-0.000107,-0.001135,0.000890,-0.001166,0.001160,0.001224,0.001092,0.001118,...,162.862503,162.880005,162.919998,162.919998,163.860001,163.860001,163.860001,163.860001,163.860001,163.860001
7,-0.000571,-0.001522,0.001332,-0.001057,0.000317,-0.001162,0.000976,0.000983,0.000984,0.000925,...,236.399994,236.399994,236.574997,236.574997,237.699997,237.699997,237.699997,237.699997,237.699997,237.699997
8,-0.002910,-0.000340,0.000678,0.000227,0.000491,-0.000264,0.000967,0.000827,0.000828,0.000847,...,132.259995,132.259995,132.259995,132.259995,132.945007,133.080002,133.080002,133.080002,133.080002,133.080002
9,-0.000539,0.001347,0.000270,0.001890,-0.002833,0.001078,0.001468,0.001480,0.001443,0.001440,...,37.005001,37.005001,37.005001,37.070000,37.334999,37.334999,37.334999,37.334999,37.334999,37.334999


In [139]:
#request_dict

In [110]:
ep_url = 'https://modelservice.ml-6e89be47-314.loki-cdp.a465-9q4k.cloudera.site/model'

In [111]:
r4 = requests.post(ep_url, data=json.dumps(request_dict),headers={'Content-Type': 'application/json'})

In [112]:
r4.json()

{'success': False,
 'ReplicaID': 'rf-new-revision-6-6-698b7569-q4lw6',
 'Size': 200,
 'StatusCode': 400}

**Multiple Inputs**

In [52]:
input_data_mult = df[cols].to_dict('records')

In [56]:
request_dict_mult = {"request":input_data_mult}

BackDict = {"accessKey":"mytbobsrpwzf5nhnqg32nsuqg06n0x7c"}
BackDict.update(request_dict_mult)
request_dict_mult=BackDict
request_dict_mult

In [140]:
r5 = requests.post(ep_url, data=json.dumps(request_dict_mult),headers={'Content-Type': 'application/json'})

In [141]:
r5.json()

{'success': False,
 'ReplicaID': 'rf-v2-5-5-58b785b848-dgcxc',
 'Size': 112,
 'StatusCode': 400}

In [142]:
request_dict_mult.keys()

dict_keys(['accessKey', 'request'])

### Trying a different approach to encoding dictionary for multiple inputs

In [83]:
#request_dict_mult['request']
#input_data_mult = df[cols].to_dict('series')
#input_data_mult = df[cols].to_json()

In [94]:
# creating my own damn dictionary
input_data_mult = dict()
for col in cols:
    input_data_mult[col] = df[col].to_list()    

In [97]:
#input_data_mult

In [98]:
request_dict_mult = {"request":input_data_mult}
BackDict = {"accessKey":"mytbobsrpwzf5nhnqg32nsuqg06n0x7c"}
BackDict.update(request_dict_mult)
request_dict_mult=BackDict
#request_dict_mult

In [103]:
# request_dict_mult['request']['return_lag_1'].to_list()
#request_dict_mult

In [104]:
json.dumps(request_dict_mult)

'{"accessKey": "mytbobsrpwzf5nhnqg32nsuqg06n0x7c", "request": {"return_lag_1": [-0.0021425126979011, -0.0022490930906144, -0.0001389282088331, -0.000327179097279, -0.0007702210963869, -0.0014033376973431, -0.0003079100537979, -0.0005713146543069, -0.0029099585021106, -0.0005386301292891], "return_lag_2": [0.0011643342299438, 0.0003135160193135, -0.0008329524237045, 0.0013092966822597, -0.0006064255431239, -0.0014013711001281, -0.0027207436866113, -0.0015219763920018, -0.0003395639922771, 0.0013473254209539], "return_lag_3": [0.0005597566804979, 0.0008365767857185, -0.0009709369976002, 0.0022955118273249, 0.0010262245993342, 0.0008278720489438, -0.0001074580728349, 0.0013316106843146, 0.0006784368987151, 0.0002696418989847], "return_lag_4": [0.0018895733736993, 0.0003138768558832, 0.0001385720187442, 0.0023007305655657, -0.0019911349433234, 0.0021045371781046, -0.0011351459559232, -0.0010569708475889, 0.0002273234368903, 0.00188984108876], "return_lag_5": [-0.0014294624692999, 0.0010469

In [100]:
r6 = requests.post(ep_url, data=json.dumps(request_dict_mult),headers={'Content-Type': 'application/json'})

In [101]:
r6.json()

{'success': False,
 'ReplicaID': 'rf-v2-5-5-58b785b848-dgcxc',
 'Size': 112,
 'StatusCode': 400}

### Model Prediction

In [12]:
result = pipe_rf.predict(df[cols])

In [29]:
result

array([-1,  1, -1,  1,  1,  1,  1,  1, -1, -1])